In [1]:
#-------------自动求导
import torch

x = torch.randn((4, 4), requires_grad=True)
y = 2 * x
z = y.sum()
z

tensor(2.0287, grad_fn=<SumBackward0>)

In [ ]:
x

In [ ]:
y

In [ ]:
z

In [ ]:
z.backward()

In [ ]:
x.grad

In [6]:
#------------------容器
# 方法1
import torch.nn as nn
model = nn.Sequential()
model.add_module('fc1', nn.Linear(3,4))
model.add_module('fc2', nn.Linear(4,2))
model.add_module('output', nn.Softmax(2))

In [7]:
#以参数列表的方式来定义
import torch.nn as nn
model_c = nn.Sequential(
                    nn.Linear(28*28, 32),
                    nn.ReLU(),
                    nn.Linear(32, 10),
                    nn.Softmax(dim=1),
                        )
print(model_c)
print("\n显示网络模型参数")
print(model_c.parameters)

print("\n定义神经网络样本输入")
x_input = torch.randn(2, 28, 28, 1)
print(x_input.shape)
print("\n使用神经网络进行预测")
y_pred = model_c.forward(x_input.view(x_input.size()[0], -1))
print(y_pred)

Sequential(
  (0): Linear(in_features=784, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=10, bias=True)
  (3): Softmax(dim=1)
)

显示网络模型参数
<bound method Module.parameters of Sequential(
  (0): Linear(in_features=784, out_features=32, bias=True)
  (1): ReLU()
  (2): Linear(in_features=32, out_features=10, bias=True)
  (3): Softmax(dim=1)
)>

定义神经网络样本输入
torch.Size([2, 28, 28, 1])

使用神经网络进行预测
tensor([[0.0939, 0.0785, 0.1250, 0.0760, 0.0913, 0.0837, 0.1128, 0.0987, 0.1093,
         0.1309],
        [0.0797, 0.0811, 0.1232, 0.0669, 0.1504, 0.0632, 0.0911, 0.1549, 0.0669,
         0.1225]], grad_fn=<SoftmaxBackward0>)


In [9]:
print(net)

NameError: name 'net' is not defined

In [3]:
"""
import torch.nn as nn
#以字典的方式实例化
model = nn.Sequential(OrderedDict([('h1', nn.Linear(28*28, 32)),
                                ('relu1', nn.ReLU()),
                                 ('out', nn.Linear(32, 10)),
                                ('softmax', nn.Softmax(dim=1))]))
print(model)
print("\n定义神经网络样本输入")
x_input = torch.randn(2, 28, 28, 1)
print(x_input.shape)
print(x_input.shape)
print("\n使用神经网络进行预测")
y_pred = model.forward(x_input.view(x_input.size()[0], -1))
print(y_pred)
"""

'\nimport torch.nn as nn\n#以字典的方式实例化\nmodel = nn.Sequential(OrderedDict([(\'h1\', nn.Linear(28*28, 32)),\n                                (\'relu1\', nn.ReLU()),\n                                 (\'out\', nn.Linear(32, 10)),\n                                (\'softmax\', nn.Softmax(dim=1))]))\nprint(model)\nprint("\n定义神经网络样本输入")\nx_input = torch.randn(2, 28, 28, 1)\nprint(x_input.shape)\nprint(x_input.shape)\nprint("\n使用神经网络进行预测")\ny_pred = model.forward(x_input.view(x_input.size()[0], -1))\nprint(y_pred)\n'

In [4]:
import torch
model2 = nn.Sequential(
         nn.Conv2d(1, 20, 5),
         nn.ReLU(),
         nn.Conv2d(20, 64, 5),
         nn.ReLU()
         )

In [5]:
model3 = nn.ModuleList([nn.Linear(3,4), nn.ReLU(), nn.Linear(4,2)])

In [14]:
#Linear模型
import torch
class Linear(torch.nn.Module):
    def __init__(self, in_features, out_features, bias = True):
        super(Linear, self).__init__()
        self.weight = torch.nn.Parameter(torch.randn(out_features, in_features))
        if bias:
            self.bias = torch.nn.Parameter(torch.randn(out_features))
    def forward(self, x):
        x = x.mm(self.weight)
        if self.bias:
            x + x + self.bias.expand_as(x)
        return x

if __name__ == '__main__':
    # train for mnist
    net = Linear(3,2)
    x = net.forward
    print('11',x)


11 <bound method Linear.forward of Linear()>


In [ ]:
#mnistnet网络构建
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

class Model:
    def __init__(self, net, cost, optimist):
        self.net = net
        self.cost = self.create_cost(cost)
        self.optimizer = self.create_optimizer(optimist)
        pass
    
    def create_cost(self, cost):
        support_cost = {
            'CROSS_ENTROPY': nn.CrossEntropyLoss(),
            'MSE': nn.MSELoss()
        }
        return support_cost[cost]
    
    def create_optimizer(self, optimist, **rests):
        support_optim = {
            'SGD': optim.SGD(self.net.parameters(), lr = 0.1, **rests),
            'ADAM': optim.Adam(self.net.parameters(), lr = 0.01, **rests),
            'RMSP': optim.RMSprop(self.net.parameters(), lr = 0.001, **rests)
        }
        return support_optim[optimist]
    
    def train(self, train_loader, epoches=3):
        for epoch in range(epoches):
            running_loss = 0.0
            for i, data in enumerate(train_loader, 0):
                inputs, labels = data
                self.optimizer.zero_grad()
                outputs = self.net(inputs)
                loss = self.cost(outputs, labels)
                loss.backward()
                self.optimizer.step()
                running_loss += loss.item()
                if i % 100 == 0:
                    print('[epoch %d, %.2f%%] loss: %.3f' %
                         (epoch + 1, (i + 1)*1./len(train_loader), running_loss / 100))
                    running_loss = 0.0
            print('Finished Training')
    
    def evaluate(self, test_loader):
        print('Evaluating ...')
        correct = 0
        total = 0
        with torch.no_grad():
            for data in test_loader:
                images, labels = data
                outputs = self.net(images)
                predicted = torch.argmax(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

def mnist_load_data():
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize([0,], [1,])])

    trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                            download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                              shuffle=True, num_workers=2)

    testset = torchvision.datasets.MNIST(root='./data', train=False,
                                           download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=32,shuffle=True, num_workers=2)
    return trainloader, testloader

class MnistNet(torch.nn.Module):
    def __init__(self):
        super(MnistNet, self).__init__()
        self.fc1 = torch.nn.Linear(28*28, 512)
        self.fc2 = torch.nn.Linear(512, 512)
        self.fc3 = torch.nn.Linear(512, 10)
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x), dim=1)
        return x

if __name__ == '__main__':
    net = MnistNet()
    model = Model(net, 'CROSS_ENTROPY', 'RMSP')
    train_loader, test_loader = mnist_load_data()
    model.train(train_loader)
    model.evaluate(test_loader)

[epoch 1, 0.00%] loss: 0.023
[epoch 1, 0.05%] loss: 1.805
[epoch 1, 0.11%] loss: 1.587
[epoch 1, 0.16%] loss: 1.564
[epoch 1, 0.21%] loss: 1.555
[epoch 1, 0.27%] loss: 1.541
[epoch 1, 0.32%] loss: 1.537
[epoch 1, 0.37%] loss: 1.531
[epoch 1, 0.43%] loss: 1.529
[epoch 1, 0.48%] loss: 1.528
[epoch 1, 0.53%] loss: 1.530
[epoch 1, 0.59%] loss: 1.521
